## 1.此程式:
爬取某座城市的所有Hotel name，並存成.npy檔案

In [1]:
from selenium import webdriver #The Selenium package we'll need
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import csv #This package lets us save data to a csv file
import re
import time #This package lets us pause execution for a bit
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import random
# 靜態爬蟲
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import json


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-images')
options.add_argument('--disable-gpu')
# options.add_argument("--disable-javascript")
# 無痕模式
options.add_argument('--incognito')

In [4]:
# driver setting
# url = "https://www.tripadvisor.com/Hotels-g60763-New_York_City_New_York-Hotels.html" # New York
# url = "https://www.tripadvisor.com/Hotels-g34515-Orlando_Florida-Hotels.html" # Orlando
# url = "https://www.tripadvisor.com/Hotels-g35805-Chicago_Illinois-Hotels.html" # Chicago
url = "https://www.tripadvisor.com/Hotels-g32655-Los_Angeles_California-Hotels.html" # Chicago
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.maximize_window()
driver.get(url)
# driver.implicitly_wait(10)

In [5]:
import json

# print(driver.page_source)
Path('source.html').write_text(driver.page_source)

hotels_data = driver.find_element(By.XPATH, "//div[@data-hotels-data]")
print(hotels_data.get_attribute('data-hotels-data'))

hotels_dict = json.loads(hotels_data.get_attribute("data-hotels-data"))
hotels = hotels_dict['hotels']
hotel_list = [{'hotel_name': h['name'], 'url': h['detailUrl'], 'num_reviews': h['numReviews']} for h in hotels]
len(hotel_list)

{"geoId":32655,"hotels":[{"entityType":"hotel","id":119421,"name":"Los Angeles Athletic Club","detailUrl":"/Hotel_Review-g32655-d119421-Reviews-Los_Angeles_Athletic_Club-Los_Angeles_California.html","numReviews":560,"bubbleRating":40,"geoPoint":{"latitude":34.04664611816406,"longitude":-118.2552490234375},"thumbnail":null,"popIndexText":"#47 of 419 Los Angeles hotels","accommodationCategory":null,"offers":[],"sponsoredListing":{"impression_key":"84e5fc099b13451e89d9e0cf903e333d","tt_placement":"Hotels_MapDetail","opportunity_uid":"NA","sponsored_type":"DISCOVERY_STANDARD","sponsored_location_source":"TRIP_ADS","sl_line_item_id":21898795,"trip_ads_id":21419614,"trip_ads_select_request_guid":"8734cfcc-0281-4681-8ed9-b900bb48f4e1","is_backfill":true,"candidate_location_ids":[119421],"placement_slot_name":"HOTELS_SLOT_0","tracking_guid":"ab567ee4-f5c1-4c32-b37f-025dda62292f","gosox":"ei=UqpMqTtqWbJQuMb9ABQCSePqDp3lSw2bHWarjqN6fCiNysZkai9DOHTGXuVodWA9nX9KEAn8uc1iC6CrqxOBhsBlUAZS7FBh7vAu|ciy

36

## Hotel list scraping

In [6]:
# 先爬取頁數
driver.get(url)

element = driver.find_element(By.XPATH, '//span[@class="highlight"]')

# 使用正則表達式提取數字部分
number_match = re.search(r'\d+', element.get_attribute("outerHTML"))

if number_match:
    N_PROPERTIES = int(number_match.group())
    print(f'There are {N_PROPERTIES} properties')
else:
    print('No properties found.')

PER_PAGE = 30
N_PAGES = math.ceil(N_PROPERTIES / PER_PAGE)
print(f'There are {N_PAGES} different pages')

There are 964 properties
There are 33 different pages


In [7]:
def get_id_from_url(url):
  # Split URL by -g to divide it before the ID
  prefix, suffix = url.split('-g', maxsplit=1)

  # Divide the URL after the ID (first dash)
  id, slug = suffix.split('-', maxsplit=1)

  return int(id)

In [8]:
# Path('source.html').write_text(driver.page_source)

In [9]:
def get_listing_url(page, base_url=url, per_page=PER_PAGE):
  assert page >= 0

  id = get_id_from_url(base_url)
  
  if page == 0:
    return url
  return url.replace(f'-g{id}-', f'-g{id}-oa{page * per_page}-')

In [10]:
# 爬取這個城市所有飯店的URL

time.sleep(2)

hotel_raw_data = []
# See all hotel list

driver.find_element(By.XPATH, '//button[@class="rmyCe _G B- z _S c Wc wSSLS pexOo sOtnj"]').click()
for i in tqdm(range(N_PAGES)):
    hotel_list_url = get_listing_url(i)
    # Path('source.html').write_text(driver.page_source)
    driver.get(hotel_list_url)
    
    hotels_data = driver.find_element(By.XPATH, "//div[@data-hotels-data]")
    # print(hotels_data.get_attribute('data-hotels-data'))
    hotels_dict = json.loads(hotels_data.get_attribute("data-hotels-data"))
    hotels = hotels_dict['hotels']
    hotel_list = [{'Hotel_locID': h['id'], 'Hotel_name': h['name'], 'Url': h['detailUrl'], 'Num_reviews': h['numReviews']} for h in hotels]
    hotel_raw_data = [*hotel_raw_data, *hotel_list]
    

100%|██████████| 33/33 [01:27<00:00,  2.64s/it]


In [9]:
hotel_raw_data

[{'Hotel_locID': 87610,
  'Hotel_name': 'Holiday Inn Chicago – Midway Airport S',
  'Url': '/Hotel_Review-g35805-d87610-Reviews-Holiday_Inn_Chicago_Midway_Airport_S-Chicago_Illinois.html',
  'Num_reviews': 254},
 {'Hotel_locID': 9145578,
  'Hotel_name': 'LondonHouse Chicago, Curio Collection by Hilton',
  'Url': '/Hotel_Review-g35805-d9145578-Reviews-LondonHouse_Chicago_Curio_Collection_by_Hilton-Chicago_Illinois.html',
  'Num_reviews': 3803},
 {'Hotel_locID': 114581,
  'Hotel_name': 'Swissotel Chicago',
  'Url': '/Hotel_Review-g35805-d114581-Reviews-Swissotel_Chicago-Chicago_Illinois.html',
  'Num_reviews': 6440},
 {'Hotel_locID': 87603,
  'Hotel_name': 'The Royal Sonesta Chicago Downtown',
  'Url': '/Hotel_Review-g35805-d87603-Reviews-The_Royal_Sonesta_Chicago_Downtown-Chicago_Illinois.html',
  'Num_reviews': 262},
 {'Hotel_locID': 111486,
  'Hotel_name': 'Hampton Inn & Suites Chicago-Downtown',
  'Url': '/Hotel_Review-g35805-d111486-Reviews-Hampton_Inn_Suites_Chicago_Downtown-Chicag

In [11]:
#去除評論數過少的飯店
hotel_data = []

for data in hotel_raw_data:
    if data['Num_reviews'] >= 30:
        hotel_data.append(data)

In [12]:
len(hotel_data)

502

In [23]:
# driver.find_element(By.XPATH, '//button[@class="rmyCe _G B- z _S c Wc wSSLS pexOo sOtnj"]').click()
# for i in tqdm(range(N_PAGES)):
#     # give the DOM time to load
#     time.sleep(2)
    
#     blocks = driver.find_elements(By.XPATH, '//div[@class="NXAUb _T"]')
#     for block in blocks:
#         print(block)
    
#     driver.find_element(By.XPATH, '//a[@class="BrOJk u j z _F wSSLS tIqAi unMkR"]').click()
        
# driver.quit()

In [13]:
#儲存
hotel_data_store=np.array(hotel_data)
np.save('/home/hao/Code/variable/hotel_data_la_store.npy', hotel_data_store) #儲存為.npy格式

In [13]:
#讀取
hotel_data_read=np.load('/home/hao/Code/variable/hotel_data_chicago_store.npy', allow_pickle=True)
hotel_data=hotel_data_read.tolist()